In [1]:
import sys 
sys.path.append("../")

from mps_utils import * 

Infidelity Heatmap

In [2]:
sigma = 0.1 
mu = 0.5
f = lambda x: 2/(np.sqrt(3 * sigma) * (np.pi)**(1/4)) * (1 - ((x-mu)/sigma)**2 ) * np.exp(- (x-mu)**2/(2*sigma**2))

df = lambda x: -4/(np.sqrt(3 * sigma) * (np.pi)**(1/4)) * (x-mu)/(sigma**2) * np.exp(- (x-mu)**2/(2*sigma**2)) - (x-mu)/(sigma**2) * f(x)

def fidelity(m, n):
    l = 2**m
    x_coarse = np.linspace(0,1,l+1)
    poly_list = get_polys(x_coarse,f,df)  
    
    N = 2**n
    x = np.linspace(0, 1 - 1/N, N)
    
    approx  = []
    for idx, v in enumerate(poly_list):
        xi = x[idx * N // l : (idx + 1) * N // l ]
        approx.append(apply(v, xi))
    
    approx_vec = np.hstack(approx)
    true_vec = f(x)
    
    return 1 - np.dot(approx_vec, true_vec)**2/(np.dot(true_vec, true_vec)**2)

# Define the ranges
m_values = [3, 4, 5, 6, 7]
n_values = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# Build matrix of fidelities
data = np.zeros((len(m_values), len(n_values)))
for i, m in enumerate(m_values):
    for j, n in enumerate(n_values):
        if m < n:
            data[i, j] = fidelity(m, n)
        else:
            data[i, j] = np.nan   # mask invalid entries

data_dict = {
    'm_values':m_values,
    'n_values':n_values,
    'data':data,
}

np.save('../data/fidelity_heatmap.npy', np.array([data_dict]), allow_pickle=True)


Poly Approx Plot

In [3]:
# Parameters
n, m = 10, 6
l = 2**m
N = 2**n

# Shifted/scaled functions
f_shift_scale  = lambda x: f(x/2 + 1/(2 * N))
df_shift_scale = lambda x: 0.5 * df(x/2 + 1/(2 * N))

# Grids
x_coarse = np.linspace(0, 1, l + 1)
x_fine   = np.linspace(0, 1 - 1/N, N)
x_shift  = x_fine + 1/(2 * N)

# Piecewise polynomials
poly_list = get_polys(x_coarse, f_shift_scale, df_shift_scale)


approx_segments = []
error_segments  = []

for idx, v in enumerate(poly_list):
    xi = x_fine[idx * N // l : (idx + 1) * N // l + 1]
    yi = apply(v, xi)

    x_plot     = 0.5 * xi + 1 / (2 * N)
    x_reflect  = 1 - x_plot

    approx_segments.append({
        "x": x_plot,
        "y": yi,
        "x_reflect": x_reflect,
    })

    error_segments.append({
        "x": x_plot,
        "y": f(x_plot) - yi,
        "x_reflect": x_reflect,
        "y_reflect": f(x_reflect) - yi,
    })

# Reference function data
ref_data = {
    "x": x_shift,
    "y": f(x_shift),
}

# Axis limits
ymin, ymax = f(x_fine).min(), f(x_fine).max()

np.save('../data/polyapprox',
    np.array([*approx_segments,
    *error_segments,
    ref_data,
    {'min':ymin, 'max':ymax}    
    ]),
    allow_pickle=True
)

Truncation Plot

In [4]:
n = 10
N = 2**n
x = np.linspace(0, 1 - 1/N, N) + 1/(2*N)
true_state = f(x) / np.linalg.norm(f(x))
f_shift_scale  = lambda x : f(x/2 + 1/(2 * N))
df_shift_scale = lambda x : 1/2 * df(x/2 + 1/(2 * N))


def get_infidelity(m, trunc):
    l = 2**m
    x_coarse = np.linspace(0,1,l+1)
    poly_list = get_polys(x_coarse,f_shift_scale,df_shift_scale)  
    
    M_list = []
    for i,p in enumerate(poly_list):
        reg = [i//2**_%2 for _ in range(m)[::-1]]
        M = MPS_poly(n-1,p,reg)
        M_list.append(M)

    M_full = (reduce(lambda a,b: MPS_sum(a,b),M_list))

    M_trunc = trunc_mps(M_full, trunc)
    state = get_state(M_trunc)
    M_state = np.concatenate([state, state[::-1]])/np.sqrt(2)
    
    
    return 1 - (M_state.dot(true_state))**2

l = []
for m in range(4,8):
    x_values = list(range(2, 13))
    y_values = [get_infidelity(m, trunc) for trunc in range(2, 13)]
    
    l.append({
        'x_values':x_values, 
        'y_values':y_values
    })

np.save('../data/truncdata',
    l,
    allow_pickle=True
)

In [5]:
l = 2**m
x_coarse = np.linspace(0,1,l+1)
poly_list = get_polys(x_coarse,f_shift_scale,df_shift_scale)  

M_list = []
for i,p in enumerate(poly_list):
    reg = [i//2**_%2 for _ in range(m)[::-1]]
    M = MPS_poly(n-1,p,reg)
    M_list.append(M)

M_full = (reduce(lambda a,b: MPS_sum(a,b),M_list))

M_trunc = trunc_mps(M_full, 6)

Training Plot

In [6]:
num_layers = 6
opt_layers, results = optimize_mps_circuit(M_trunc, num_layers, r=1e-2, opt_iters=10000)

layers = []
while len(layers) < num_layers:
    layers = get_next_layer(layers, M_trunc)
    
    exact_state = f(x) / np.linalg.norm(f(x))

mpd_infid, grad_infid = [], []
mpd_states, grad_states = [], []

for i in range(num_layers):
    state = zero_mps(n-1)
    for l in layers[num_layers-1 - i:]:
        state = entangle_layer(state, l)
    state = get_state(state)
    state = np.concatenate([state, state[::-1]])/np.sqrt(2)
    mpd_infid.append(1 - abs(np.dot(exact_state, state))**2)
    mpd_states.append(state)
    
    # Bottom row: Gradient (cool pastel)
    state = zero_mps(n-1)
    temp_layers = opt_layers[i]
    for l in temp_layers:
        state = entangle_layer(state, l)
    state = get_state(state)
    state = np.concatenate([state, state[::-1]])/np.sqrt(2)
    grad_infid.append(1 - abs(np.dot(exact_state, state))**2)
    grad_states.append(state)
    
np.save(
    '../data/mps_opt_data',
    [{'opt_layers': opt_layers, 'results': results, 'layers':layers, 'mpd_states':mpd_states, 'grad_states':grad_states,
       'mpd_infid':mpd_infid, 'grad_infid':grad_infid,'x':x}],
    allow_pickle=True
)

----------------------------------------------
----------------------------------------------
----------------------------------------------
----------------------------------------------
----------------------------------------------
----------------------------------------------
